# Dynet segmentation with tf fold
![animation](../../fold/tensorflow_fold/g3doc/animation.gif)  

In [2]:
#just a bunch of fun
import numpy as np
import six
from multiprocessing import Process, Queue
import time
import data
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
import tensorflow_fold as td
from conv_lstm_cell import *

# params
EMBEDDING_SIZE = 64
SEP = "|"
BATCH_SIZE = 100
data_dir = "/home/moon/data/"

#our alphabet

vocabulary = data.vocabulary(data_dir + 'vocabulary')
vsize=len(vocabulary)
print(vocabulary)

index = lambda char: vocabulary.index(char)
char = lambda i: vocabulary[i]



[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'C', 'D', 'E', 'G', 'L', 'N', 'S', 'T', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '²', '³', '¹', 'á', 'é', 'í', 'ó', 'ö', 'ú', 'ü', 'ő', 'ű']


In [3]:
class reader():
    def __init__(self, folder, mode="bidirectional", qsize=10000, start=True):
        self.data_dir = folder
        self.qsize= qsize
        self.pool = dict()
        self.data = dict()
        self.datasets = ["train", "test", "validation"]
        if mode == "bidirectional":
            convert_fn = self.bidirectional_sentence_reader
        else:
            convert_fn = self.sentence_reader
            
        for dataset in self.datasets:
            self.data[dataset] = Queue(qsize)
            self.pool[dataset] = Process(target=convert_fn, args=(self.data[dataset], self.data_dir+dataset))
        if start == True:
            self.start()
            
    def sentence_reader(self, queue, file):
        """
        read sentences from the data format setence: word\tword\n.....\t\n
        """
        while True:
            with open(file) as f:
                while True:
                    try:
                        sentence = []
                        while True:
                            line = f.readline()[:-1].split('\t')
                            if line[0] != "":
                                sentence.append(line)
                            else:
                                break
                        sent = " ".join([word[0] for word in sentence])
                        segmented = " ".join([word[1].replace(" ","|") for word in sentence])
                        tags = []
                        last_char = "_"
                        for char in segmented:
                            if char != "|":
                                tags.append(0 if last_char!="|" else 1)
                            last_char = char
                        queue.put((sent, tags))
                    except e:
                        print(e)
                        
    def bidirectional_sentence_reader(self, queue, file):
        """
        read sentences from the data format setence: word\tword\n.....\t\n
        """
        while True:
            with open(file) as f:
                while True:
                    try:
                        sentence = []
                        while True:
                            line = f.readline()[:-1].split('\t')
                            if line[0] != "":
                                sentence.append(line)
                            else:
                                break
                        sent = " ".join([word[0] for word in sentence])
                        segmented = " ".join([word[1].replace(" ","|") for word in sentence])
                        tags = []
                        last_char = "_"
                        for char in segmented:
                            if char != "|":
                                tags.append(0 if last_char!="|" else 1)
                            last_char = char
                        for i in range(1, len(sent)-1):
                            forward, backward = sent[:i], sent[i:][::-1]
                            queue.put(([self.onehot(forward), self.onehot(backward)], tags[i]))
                    except:
                        print("err")
                        
    def start(self):
        for dataset in self.datasets:
            self.pool[dataset].start()
            
    def stop(self):
        for dataset in self.datasets:
            self.pool[dataset].terminate()
            
    def get(self,dataset):
        if dataset in self.datasets:
            return self.data[dataset].get()
        else:
            raise KeyError
            
    def onehot(self, string):
        onehot = np.zeros([len(string),vsize])
        onehot[np.arange(len(string)), np.array([index(char) for char in string])]=1
        return [onehot[i,:] for i in range(len(onehot))]
            
store = reader("/home/moon/data/")


err
err
err


## helper functions

In [4]:
def params_info():
    total_parameters = 0
    for variable in tf.trainable_variables():
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        print(variable.name, shape)
        # print(len(shape))
        variable_parametes = 1
        for dim in shape:
            # print(dim)
            variable_parametes *= dim.value
        print("\tparams: ", variable_parametes)
        total_parameters += variable_parametes
    print(total_parameters)
    return total_parameters

def onehot(string):
    onehot = np.zeros([len(string),vsize])
    onehot[np.arange(len(string)), np.array([index(char) for char in string])]=1
    return [onehot[i,:] for i in range(len(onehot))]


In [5]:
#cell = td.ScopedLayer(tf.contrib.rnn.BasicLSTMCell(num_units=16), 'char_cell')


In [6]:
def bidirectional_conv_LSTM():
    convlstm = Conv1DLSTMCell(input_shape=[vsize,1], output_channels=8, kernel_shape=[5])
    conv_lstm_cell_1d = td.ScopedLayer(convlstm)

    bidir_conv_lstm = td.Composition()
    with bidir_conv_lstm.scope():

        forward = td.Identity().reads(bidir_conv_lstm.input[0])
        backward = td.Identity().reads(bidir_conv_lstm.input[1])

        forw = (td.RNN(conv_lstm_cell_1d) >>
                td.GetItem(1) >>
                td.GetItem(0) >>
                td.Function(lambda rnn_outs: tf.contrib.layers.flatten(rnn_outs))).reads(forward)

        backw = (td.RNN(conv_lstm_cell_1d) >>
                 td.GetItem(1) >>
                 td.GetItem(0) >>
                 td.Function(lambda rnn_outs: tf.contrib.layers.flatten(rnn_outs))).reads(backward)

        rnn_outs = td.Concat().reads(forw,backw)
        bidir_conv_lstm.output.reads(rnn_outs)
    return bidir_conv_lstm >> td.FC(1)


def FCNN():
    return td.FC(50) >> td.FC(1)# >> td.Function(lambda xs: tf.squeeze(xs, axis=1))

data = td.Record((td.Map(
                        td.Vector(vsize) >>
                        td.Function(lambda x: tf.reshape(x, [-1,vsize,1]))),
                  td.Map(
                        td.Vector(vsize) >>
                        td.Function(lambda x: tf.reshape(x, [-1,vsize,1])))))
bidir =  data >> bidirectional_conv_LSTM()
#fc = FCNN()
blk = bidir# >> fc 

In [7]:
compiler = td.Compiler.create((blk, td.Scalar()))
model_output, target = compiler.output_tensors
loss = tf.nn.l2_loss(model_output - target)
opt = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = opt.minimize(loss)
sess.run(tf.global_variables_initializer())



/home/moon/Envs/tf1/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:95: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [9]:
sess.run(tf.global_variables_initializer())

In [10]:
for i in range(1000):
    loff, _ = sess.run([loss, train_op], compiler.build_feed_dict([store.get("train") for _ in range(100)]))
    print(loff)

246.458
685.056
482.552
474.256
659.585
723.075
579.995
575.374
399.105
677.9
712.975
578.203
611.093
558.325
438.797
607.192
420.571
398.178
249.021
278.413
301.203
549.477
609.731
807.9
699.292
518.971
787.421
958.979
379.397
549.813
373.706
495.523
704.662
375.872
217.173
255.295
456.508
492.899
644.405
417.098
458.026
534.276
415.951
650.183
373.746
648.887
687.689
647.55
532.74
533.12
535.207
717.32
390.821
605.364
676.95
386.572
719.99
569.853
346.084
768.677
457.829
494.685
571.354
532.755
681.657
458.127
570.161
605.603
531.325
453.662
384.703
416.872
335.686
453.868
332.347
332.088
332.563
372.147
288.596
578.86
415.014
370.607
497.043
415.9
415.561
586.308
668.586
412.488
663.968
576.586
412.158
656.506
455.786
453.899
263.114
414.912
606.766
568.38
572.172
492.19
492.18
605.434
493.662
492.859
717.415
608.954
609.326
936.895
682.646
880.88
573.099
444.294
575.452
745.399
643.775
521.621
445.885
535.976
606.432
638.584
604.479
828.461
493.298
603.297
607.198
374.968
679.651
7

KeyboardInterrupt: 